In [99]:
import nltk
nltk.download('punkt') 
from nltk.tokenize import word_tokenize
def tokenizer(text):
    return word_tokenize(text)
data['Tokens']=data['Text'].apply(tokenizer)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\balui\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
data

,Text,Label,Tokens
0,damn thought they had strict gun law germany,0,"[damn, thought, they, had, strict, gun, law, g..."
1,not care about what stand for anything its con...,0,"[not, care, about, what, stand, for, anything,..."
2,not group idea lol,0,"[not, group, idea, lol]"
3,not just america,0,"[not, just, america]"
4,the dog spectacular dancer considering has two...,0,"[the, dog, spectacular, dancer, considering, h..."
...,...,...,...
17591,find rat nicer and cleaner than most chinese,1,"[find, rat, nicer, and, cleaner, than, most, c..."
17592,check out this niggar they hit thing like wild...,1,"[check, out, this, niggar, they, hit, thing, l..."
17593,this country has become absolute shamble the a...,0,"[this, country, has, become, absolute, shamble..."
17594,aged 16 antisemitism bad aged 18 antisemitism ...,1,"[aged, 16, antisemitism, bad, aged, 18, antise..."


In [125]:
from sklearn.feature_extraction.text import TfidfVectorizer
texts = data['Text'].values
labels = data['Label'].values
def tfidf_embedding(texts):
    vectorizer = TfidfVectorizer(max_features=5000)
    embeddings = vectorizer.fit_transform(texts)
    return embeddings

embeddings_tfidf = tfidf_embedding(texts)
print(embeddings_tfidf)

  (0, 1874)	0.38655864797204015
  (0, 2565)	0.3369183517812025
  (0, 1964)	0.4085287030056687
  (0, 4277)	0.49396554919123437
  (0, 1970)	0.26594905432797417
  (0, 4470)	0.15796206623433257
  (0, 4484)	0.327463512592553
  (0, 1103)	0.35106624018460947
  (1, 3988)	0.46764828617011794
  (1, 4455)	0.10452774819368486
  (1, 2633)	0.18460642886856596
  (1, 939)	0.49051943994950004
  (1, 2405)	0.2687361424930761
  (1, 248)	0.2965908260733827
  (1, 1750)	0.1649905427170199
  (1, 4221)	0.3416575901596179
  (1, 4856)	0.2044196148320207
  (1, 64)	0.21152394159488963
  (1, 694)	0.29866739942996134
  (1, 3066)	0.12578228405018932
  (2, 2674)	0.5587285254775806
  (2, 2175)	0.5796117172772546
  (2, 1946)	0.548976877964676
  (2, 3066)	0.22471555236057383
  (3, 209)	0.8296004517693428
  :	:
  (17594, 8)	0.16430283962091688
  (17594, 14)	0.14253051698074895
  (17594, 1555)	0.12228340995633794
  (17594, 4482)	0.08772139759125756
  (17594, 384)	0.1023801384482584
  (17594, 3120)	0.14614376962658285
  (17

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(embeddings_tfidf, labels, test_size=0.2, random_state=42)

def train_evaluate_rf(X_train_emb, X_test_emb, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train_emb, y_train)
    y_pred = rf.predict(X_test_emb)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report

accuracy_tfidf, report_tfidf = train_evaluate_rf(X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf)
print(f'TF-IDF Accuracy: {accuracy_tfidf}')
print(f'TF-IDF Classification Report:\n{report_tfidf}')

TF-IDF Accuracy: 0.6792613636363637
TF-IDF Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.80      0.75      2094
           1       0.63      0.50      0.56      1426

    accuracy                           0.68      3520
   macro avg       0.67      0.65      0.65      3520
weighted avg       0.67      0.68      0.67      3520



In [128]:
from gensim.models import Word2Vec

def word2vec_embedding(texts):
    model = Word2Vec(texts, vector_size=100, window=5, min_count=1, workers=4)
    word_vectors = model.wv

    def get_word2vec_embeddings(text, word_vectors):
        embeddings = [word_vectors[word] for word in text if word in word_vectors]
        if embeddings:
            return np.mean(embeddings, axis=0)
        else:
            return np.zeros(100)

    embeddings = np.array([get_word2vec_embeddings(text, word_vectors) for text in texts])
    return embeddings

embeddings_w2v = word2vec_embedding(data['Tokens'])
print(embeddings_w2v)

[[-0.34244603  0.56320775 -0.13862239 ... -0.64171565  0.26037344
   0.11035831]
 [-0.54921669  0.89830345 -0.23524024 ... -0.73087049  0.3274487
   0.01425049]
 [-0.55216449  0.81007117 -0.21898635 ... -0.89701265  0.2908527
   0.02773679]
 ...
 [-0.30935851  0.54940945 -0.15352981 ... -0.64945304  0.41857535
   0.0172969 ]
 [-0.46842033  0.73636758 -0.18529943 ... -0.61124378  0.25507087
  -0.02185149]
 [-0.45092741  0.78913182 -0.14448845 ... -0.68255621  0.19861913
  -0.11676348]]


In [122]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
def one_hot_encoding(texts):
    vectorizer = CountVectorizer(binary=True)
    embeddings = vectorizer.fit_transform(texts)
    return embeddings

embeddings_one_hot = one_hot_encoding(texts)
X_train_one_hot, X_test_one_hot, y_train_one_hot, y_test_one_hot = train_test_split(embeddings_one_hot, labels, test_size=0.2, random_state=42)

print(embeddings_one_hot)

  (0, 4315)	1
  (0, 17087)	1
  (0, 17033)	1
  (0, 7589)	1
  (0, 16345)	1
  (0, 7542)	1
  (0, 9660)	1
  (0, 7162)	1
  (1, 11708)	1
  (1, 2841)	1
  (1, 432)	1
  (1, 18571)	1
  (1, 16163)	1
  (1, 6681)	1
  (1, 1153)	1
  (1, 9044)	1
  (1, 3732)	1
  (1, 9903)	1
  (1, 16979)	1
  (1, 15319)	1
  (2, 11708)	1
  (2, 7492)	1
  (2, 8307)	1
  (2, 10035)	1
  (3, 11708)	1
  :	:
  (17594, 14863)	1
  (17594, 11962)	1
  (17594, 1655)	1
  (17594, 17080)	1
  (17594, 6028)	1
  (17594, 136)	1
  (17594, 67)	1
  (17594, 5843)	1
  (17594, 3671)	1
  (17594, 13871)	1
  (17594, 9153)	1
  (17594, 16205)	1
  (17594, 14867)	1
  (17594, 10620)	1
  (17594, 711)	1
  (17594, 84)	1
  (17594, 11347)	1
  (17594, 1122)	1
  (17594, 14409)	1
  (17595, 11708)	1
  (17595, 2122)	1
  (17595, 14463)	1
  (17595, 4683)	1
  (17595, 14867)	1
  (17595, 10648)	1


In [130]:
def term_frequency_encoding(texts):
    vectorizer = CountVectorizer()
    embeddings = vectorizer.fit_transform(texts)
    return embeddings

embeddings_tf = term_frequency_encoding(texts)
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(embeddings_tf, labels, test_size=0.2, random_state=42)
print(embeddings_tf)

  (0, 4315)	1
  (0, 17087)	1
  (0, 17033)	1
  (0, 7589)	1
  (0, 16345)	1
  (0, 7542)	1
  (0, 9660)	1
  (0, 7162)	1
  (1, 11708)	1
  (1, 2841)	1
  (1, 432)	1
  (1, 18571)	1
  (1, 16163)	1
  (1, 6681)	1
  (1, 1153)	1
  (1, 9044)	1
  (1, 3732)	1
  (1, 9903)	1
  (1, 16979)	1
  (1, 15319)	1
  (2, 11708)	1
  (2, 7492)	1
  (2, 8307)	1
  (2, 10035)	1
  (3, 11708)	1
  :	:
  (17594, 14863)	1
  (17594, 11962)	1
  (17594, 1655)	1
  (17594, 17080)	1
  (17594, 6028)	1
  (17594, 136)	1
  (17594, 67)	1
  (17594, 5843)	1
  (17594, 3671)	1
  (17594, 13871)	1
  (17594, 9153)	1
  (17594, 16205)	1
  (17594, 14867)	1
  (17594, 10620)	1
  (17594, 711)	3
  (17594, 84)	1
  (17594, 11347)	1
  (17594, 1122)	4
  (17594, 14409)	1
  (17595, 11708)	1
  (17595, 2122)	1
  (17595, 14463)	1
  (17595, 4683)	1
  (17595, 14867)	1
  (17595, 10648)	1
